# Trabalho 1 - Arquivo Main

## Preparando o Ambiente

### Importando as Bibliotecas

In [1]:
import glfw

# Solução para abrir o OpenGL no Mac
try:
    import OpenGL as ogl
    try:
        import OpenGL.GL
    except ImportError:
        print('patching Mac OS Sonoma import error')
        from ctypes import util
        orig_util_find_library = util.find_library
        def new_util_find_library( name ):
            res = orig_util_find_library( name )
            if res: return res
            return '/System/Library/Frameworks/'+name+'.framework/'+name
        util.find_library = new_util_find_library
except ImportError:
    pass

from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image
import time

# Importa as funções auxiliares
from aux import *

from objects import Object

patching Mac OS Sonoma import error


### Inicializando a janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

altura = 800
largura = 800

# Define informações da janela!
window = glfw.create_window(largura, altura, "Trabalho 1", None, None)

glfw.make_context_current(window)

### GLSL

In [3]:
# Vertex Shader
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 mat_transform;        
        
        void main(){
            gl_Position = mat_transform * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

In [4]:
# Fragmente Shader
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### GPU

In [5]:
# Requisita slots para a GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

In [6]:
# Passa o código dos shaders para a GPU
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilar Shaders

In [7]:
# Compilar Vertex Shader
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilação do Vertex Shader")


In [8]:
# Compilar Fragment Shader
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilação do Fragment Shader")

### Associar os programas compilados ao programa principal

In [9]:
# Conecta os shaders ao programa
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkar o programa

In [10]:
# Build do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Torna o programa o default
glUseProgram(program)

## Preparar os dados para enviá-los a GPU

In [11]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 5
textures = glGenTextures(qtd_texturas)

### Carregar Modelos

In [12]:
caixa = Object('caixa/caixa.obj','caixa/caixa2.jpg')
monstro = Object('monstro/monstro.obj','monstro/monstro.jpg',escala=0.1)

caixa.load()
monstro.load()


36
6108


### Enviar dados da CPU para a GPU

In [13]:
# Requisitar slot do buffer da GPU
buffer = glGenBuffers(2)

###  Enviar coordenadas de vértices para a GPU

In [14]:
vertices = np.zeros(len(Object.vertices_list), [("position", np.float32, 3)])
vertices['position'] = Object.vertices_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

###  Enviar coordenadas de textura para a GPU

In [15]:
textures = np.zeros(len(Object.textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = Object.textures_coord_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

# Inicializar Aplicação

### Exibir Janela


In [16]:
glfw.show_window(window)

### Loop Principal da Aplicação

In [17]:
glEnable(GL_DEPTH_TEST)

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)

    loc_mat_transform = glGetUniformLocation(program, "mat_transform")
    glUniformMatrix4fv(loc_mat_transform, 1, GL_FALSE,monstro.getMatTransform()) 

    #caixa.draw()
    monstro.draw()
    
    caixa.dx+=0
    caixa.dy+=0
    caixa.dz+=0.1

    glfw.swap_buffers(window)

glfw.terminate()